In [12]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [9]:
load_dotenv('.env.local')

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [ ]:
import json
data = json.load(open("reviews.json"))
data['reviews']

In [14]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model='text-embedding-3-small',
    )
    embedding = response.data[0].embedding
    processed_data.append(
        {
            "values": embedding,
            "id": review["professor"],
            "metadata": {
                "review": review["review"],
                "subject": review["subject"],
                "stars": review["stars"]
            }
        }
    )

In [15]:
processed_data[0]

{'values': [-0.0173352,
  -0.0021343397,
  -0.0061148587,
  -0.00416123,
  0.02382125,
  0.014352661,
  0.022714194,
  -0.010946834,
  0.013506088,
  -0.035946775,
  -0.009396955,
  -0.029096048,
  -0.0017126814,
  0.01142873,
  0.032378145,
  0.018963225,
  -0.023443548,
  -0.03050266,
  -0.009436028,
  0.033003304,
  0.056108225,
  -0.02373008,
  0.048606288,
  -0.034462016,
  -0.04779879,
  -0.0185725,
  0.009742097,
  0.041547175,
  0.027428951,
  0.017309153,
  0.06892403,
  -0.008276875,
  0.0125292735,
  0.015173185,
  -0.050065,
  0.047981128,
  -0.0056687803,
  0.04326637,
  0.01928883,
  -7.5346994e-05,
  0.013401895,
  0.012099476,
  -0.062568225,
  -0.023704031,
  0.028106209,
  -0.048658386,
  -0.003770504,
  -0.049491934,
  0.020005161,
  0.031049676,
  -0.050065,
  0.04454274,
  0.086272255,
  -0.024876209,
  -0.039307017,
  0.007482399,
  0.005847863,
  0.032352097,
  -0.0016589566,
  0.007749395,
  0.08163565,
  -0.006941895,
  -0.023183065,
  -0.047199678,
  -0.040244

In [18]:
index = pc.Index("rag")
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [ ]:
index.describe_index_stats()